In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
tf.disable_v2_behavior()

In [2]:
base = pd.read_csv('/content/drive/My Drive/TensorFlow_1.x_guide/linear_regression/CSV/house_prices.csv')
base.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
base.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [0]:
used_columns =['price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']

In [5]:
used_columns

['price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long']

In [0]:
base = pd.read_csv('/content/drive/My Drive/TensorFlow_1.x_guide/linear_regression/CSV/house_prices.csv', usecols=used_columns)


In [7]:
base.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045


In [0]:
scaler_X = MinMaxScaler()
base[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']] = scaler_X.fit_transform(base[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']])

In [9]:
base.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long
0,221900.0,0.090909,0.12500,0.067170,0.003108,0.0,0.0,0.0,0.5,0.500000,0.097588,0.000000,0.478261,0.000000,0.893939,0.571498,0.217608
1,538000.0,0.090909,0.28125,0.172075,0.004072,0.4,0.0,0.0,0.5,0.500000,0.206140,0.082988,0.443478,0.988089,0.626263,0.908959,0.166113
2,180000.0,0.060606,0.12500,0.036226,0.005743,0.0,0.0,0.0,0.5,0.416667,0.052632,0.000000,0.286957,0.000000,0.136364,0.936143,0.237542
3,604000.0,0.121212,0.37500,0.126038,0.002714,0.0,0.0,0.0,1.0,0.500000,0.083333,0.188797,0.565217,0.000000,0.681818,0.586939,0.104651
4,510000.0,0.090909,0.25000,0.104906,0.004579,0.0,0.0,0.0,0.5,0.583333,0.152412,0.000000,0.756522,0.000000,0.368687,0.741354,0.393688


In [0]:
scaler_y = MinMaxScaler()
base[['price']] = scaler_y.fit_transform(base[['price']])

In [0]:
X = base.drop('price', axis=1)
y = base.price

In [12]:
X.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long
0,0.090909,0.12500,0.067170,0.003108,0.0,0.0,0.0,0.5,0.500000,0.097588,0.000000,0.478261,0.000000,0.893939,0.571498,0.217608
1,0.090909,0.28125,0.172075,0.004072,0.4,0.0,0.0,0.5,0.500000,0.206140,0.082988,0.443478,0.988089,0.626263,0.908959,0.166113
2,0.060606,0.12500,0.036226,0.005743,0.0,0.0,0.0,0.5,0.416667,0.052632,0.000000,0.286957,0.000000,0.136364,0.936143,0.237542
3,0.121212,0.37500,0.126038,0.002714,0.0,0.0,0.0,1.0,0.500000,0.083333,0.188797,0.565217,0.000000,0.681818,0.586939,0.104651
4,0.090909,0.25000,0.104906,0.004579,0.0,0.0,0.0,0.5,0.583333,0.152412,0.000000,0.756522,0.000000,0.368687,0.741354,0.393688


In [13]:
y.head()

0    0.019266
1    0.060721
2    0.013770
3    0.069377
4    0.057049
Name: price, dtype: float64

In [14]:
type(X)

pandas.core.frame.DataFrame

In [15]:
type(y)

pandas.core.series.Series

In [16]:
predictors_columns = used_columns[1:17]
predictors_columns

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long']

In [0]:
columns = [tf.feature_column.numeric_column(key = c) for c in predictors_columns]

In [18]:
print(columns[10])

NumericColumn(key='sqft_basement', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3)

In [20]:
function_train = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=32, num_epochs=None, shuffle=True)

In [0]:
function_test = tf.estimator.inputs.pandas_input_fn(x=X_test, y=y_test, batch_size=32, num_epochs=10000, shuffle=False)

In [22]:
regressor = tf.estimator.LinearRegressor(feature_columns=columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpqthz3nch', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [23]:
regressor.train(input_fn=function_train, steps=10000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Please use `layer.add_weight` method instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpqthz3nch/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving c

In [24]:
metrics_train = regressor.evaluate(input_fn=function_train, steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-19T01:08:11Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpqthz3nch/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Inference Time : 24.40296s
INFO:tensorflow:Finished evaluation at 2020-05-19-01:08:35
INFO:tensorflow:Saving dict for global step 10000: average_loss = 0.00072469056, global_step = 10000, label/mean = 0.061306696, loss = 0.023190098, prediction/mean = 0.

In [25]:
metrics_test = regressor.evaluate(input_fn=function_test, steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-19T01:08:36Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpqthz3nch/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Inference Time : 24.66508s
INFO:tensorflow:Finished evaluation at 2020-05-19-01:09:01
INFO:tensorflow:Saving dict for global step 10000: average_loss = 0.0006567864, global_step = 10000, label/mean = 0.060270954, loss = 0.021017164, prediction/mean = 0.0

In [26]:
metrics_train

{'average_loss': 0.00072469056,
 'global_step': 10000,
 'label/mean': 0.061306696,
 'loss': 0.023190098,
 'prediction/mean': 0.05934777}

In [27]:
metrics_test

{'average_loss': 0.0006567864,
 'global_step': 10000,
 'label/mean': 0.060270954,
 'loss': 0.021017164,
 'prediction/mean': 0.058573842}

In [0]:
function_prevision = tf.estimator.inputs.pandas_input_fn(x= X_test, shuffle=False)

In [0]:
prevision = regressor.predict(input_fn=function_prevision)

In [30]:
list(prevision)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpqthz3nch/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'predictions': array([0.02184178], dtype=float32)},
 {'predictions': array([0.1662947], dtype=float32)},
 {'predictions': array([0.19598103], dtype=float32)},
 {'predictions': array([0.06377919], dtype=float32)},
 {'predictions': array([0.06177661], dtype=float32)},
 {'predictions': array([0.00943618], dtype=float32)},
 {'predictions': array([0.04175833], dtype=float32)},
 {'predictions': array([0.05386885], dtype=float32)},
 {'predictions': array([0.08825319], dtype=float32)},
 {'predictions': array([0.07990812], dtype=float32)},
 {'predictions': array([0.03723258], dtype=float32)},
 {'predictions': array([0.02031997], dtype=float32)},
 {'predictions': array([0.03674924], dtype=float32)},
 {'predictions': array([0.01869045], dtype=float32)},
 {'predictions': array([0.01855154], dtype=float32)},
 {'predictions': array([0.11642326], dtype=float32)},
 {'predictions': array([0.02406558], dtype=float32)},
 {'predictions': array([0.04942475], dtype=float32)},
 {'predictions': array([0.045

In [31]:
val_prevision = []
for p in regressor.predict(input_fn=function_prevision):
  val_prevision.append(p['predictions'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpqthz3nch/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [32]:
val_prevision

[array([0.02184178], dtype=float32),
 array([0.1662947], dtype=float32),
 array([0.19598103], dtype=float32),
 array([0.06377919], dtype=float32),
 array([0.06177661], dtype=float32),
 array([0.00943618], dtype=float32),
 array([0.04175833], dtype=float32),
 array([0.05386885], dtype=float32),
 array([0.08825319], dtype=float32),
 array([0.07990812], dtype=float32),
 array([0.03723258], dtype=float32),
 array([0.02031997], dtype=float32),
 array([0.03674924], dtype=float32),
 array([0.01869045], dtype=float32),
 array([0.01855154], dtype=float32),
 array([0.11642326], dtype=float32),
 array([0.02406558], dtype=float32),
 array([0.04942475], dtype=float32),
 array([0.04537345], dtype=float32),
 array([0.03308569], dtype=float32),
 array([0.07383102], dtype=float32),
 array([0.00984363], dtype=float32),
 array([0.02561044], dtype=float32),
 array([0.06755063], dtype=float32),
 array([0.03291228], dtype=float32),
 array([0.08091883], dtype=float32),
 array([0.02701457], dtype=float32),
 a

In [0]:
val_prevision = np.asarray(val_prevision).reshape(-1,1)

In [34]:
val_prevision = scaler_y.inverse_transform(val_prevision)
val_prevision

array([[ 241543.58],
       [1342997.  ],
       [1569355.2 ],
       ...,
       [ 695790.56],
       [ 464144.7 ],
       [ 492945.28]], dtype=float32)

In [0]:
y_test2 = y_test.values.reshape(-1,1)

In [36]:
y_test2.shape

(6484, 1)

In [0]:
y_test2 = scaler_y.inverse_transform(y_test2)

In [39]:
y_test2

array([[ 384000.],
       [1698890.],
       [3000000.],
       ...,
       [ 564450.],
       [ 716000.],
       [ 385000.]])

In [0]:
mae = mean_absolute_error(y_test2, val_prevision)

In [42]:
mae

124855.20520027062